## Installing Dependencies

In [1]:
%pip install numpy
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
## Importing Data
df = pd.read_csv("data/ufc_data.csv")
df

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Aljamain Sterling,Sean O'Malley,0,1,17 of 35,25 of 35,48%,71%,24 of 46,26 of 36,...,14 of 18,KO/TKO,2,0:51,5 Rnd (5-5-5-5-5),Marc Goddard,"August 19, 2023","Boston, Massachusetts, USA",UFC Bantamweight Title Bout,Sean O'Malley
1,Chris Weidman,Brad Tavares,0,0,37 of 92,70 of 115,40%,60%,39 of 94,71 of 116,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"August 19, 2023","Boston, Massachusetts, USA",Middleweight Bout,Brad Tavares
2,Neil Magny,Ian Garry,0,0,27 of 66,91 of 150,40%,60%,51 of 93,111 of 171,...,3 of 5,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,"August 19, 2023","Boston, Massachusetts, USA",Welterweight Bout,Ian Garry
3,Marlon Vera,Pedro Munhoz,0,0,141 of 251,113 of 246,56%,45%,141 of 251,113 of 246,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,"August 19, 2023","Boston, Massachusetts, USA",Bantamweight Bout,Marlon Vera
4,Zhang Weili,Amanda Lemos,1,0,163 of 217,24 of 63,75%,38%,296 of 358,29 of 68,...,0 of 0,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),Kevin MacDonald,"August 19, 2023","Boston, Massachusetts, USA",UFC Women's Strawweight Title Bout,Zhang Weili
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7267,Patrick Smith,Ray Wizard,0,0,1 of 1,1 of 1,100%,100%,1 of 1,2 of 2,...,0 of 0,Submission,1,0:58,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Patrick Smith
7268,Orlando Wiet,Robert Lucarelli,0,0,8 of 12,2 of 6,66%,33%,11 of 15,2 of 6,...,0 of 0,KO/TKO,1,2:50,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Orlando Wiet
7269,Frank Hamaker,Thaddeus Luster,0,0,2 of 3,0 of 0,66%,---,14 of 15,0 of 0,...,0 of 0,Submission,1,4:52,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Frank Hamaker
7270,Johnny Rhodes,David Levicki,0,0,11 of 17,4 of 5,64%,80%,74 of 86,95 of 102,...,1 of 1,KO/TKO,1,12:13,No Time Limit,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Open Weight Bout,Johnny Rhodes
